- tackle classification as the victim system for now
- use PEGASUS for paraphrase or, better, a comparable BART/PyTorch model as the Ex -> Ex' generator
- fine tune it with a REINFORCE approach that uses a combination of various rewards such as:
  - quality of Ex'
  - loss (y,f(Ex'))

In [1]:
%load_ext autoreload
%autoreload 2

## Setup, load models + datasets 

In [2]:
import os
import torch 
from torch.utils.data import DataLoader
from datasets import load_dataset, load_metric
import datasets, transformers
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, AutoTokenizer
from pprint import pprint
import numpy as np, pandas as pd
import scipy
from utils import *   # local script 
import pyarrow
from sentence_transformers import SentenceTransformer, util
from IPython.core.debugger import set_trace
from GPUtil import showUtilization
import seaborn as sns
from itertools import repeat
from collections import defaultdict
from IPython.display import Markdown

path_cache = './cache/'
path_results = "./results/"

seed = 420
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
devicenum = torch.cuda.current_device() if device.type == 'cuda' else -1
n_wkrs = 4 * torch.cuda.device_count()
batch_size = 64
pd.set_option("display.max_colwidth", 400)

In [3]:
# Paraphrase model (para)
para_name = "tuner007/pegasus_paraphrase"
para_tokenizer = AutoTokenizer.from_pretrained(para_name)
para_model = AutoModelForSeq2SeqLM.from_pretrained(para_name).to(device)

In [4]:
## Victim Model (VM)
vm_name = "textattack/distilbert-base-uncased-rotten-tomatoes"
vm_tokenizer = AutoTokenizer.from_pretrained(vm_name)
vm_model = AutoModelForSequenceClassification.from_pretrained(vm_name).to(device)
vm_idx2lbl = vm_model.config.id2label
vm_lbl2idx = vm_model.config.label2id
vm_num_labels = vm_model.num_labels

In [5]:
dataset = load_dataset("rotten_tomatoes")
train,valid,test = dataset['train'],dataset['validation'],dataset['test']

label_cname = 'label'
## For snli
# remove_minus1_labels = lambda x: x[label_cname] != -1
# train = train.filter(remove_minus1_labels)
# valid = valid.filter(remove_minus1_labels)
# test = test.filter(remove_minus1_labels)

# make sure that all datasets have the same number of labels as what the victim model predicts
assert train.features[label_cname].num_classes == vm_num_labels
assert valid.features[label_cname].num_classes == vm_num_labels
assert test.features[ label_cname].num_classes == vm_num_labels

train_dl = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=n_wkrs)
valid_dl = DataLoader(valid, batch_size=batch_size, shuffle=True, num_workers=n_wkrs)
test_dl = DataLoader( test,  batch_size=batch_size, shuffle=True, num_workers=n_wkrs)

Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/data/tproth/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9c411f7ecd9f3045389de0d9ce984061a1056507703d2e3183b1ac1a90816e4d)


* Which set? Start with training set. 
* get paraphrases of each example x with label y
    * how many? Start with 3
    * with what diversity? Just go with default settings for now
    * just pick the top three: we can pick better ones later
* make a new dataset of the paraphrases p1,p2,p3 for each example x
    * assumption is that they all have the same ground truth label y
* put each paraphrase through victim model (vm) to get predictions f(p1), f(p2), f(p3)
* make a function that calculates loss 
    * you will have to "shape" the reward from this function, which means you will have to try out a number of different things 
    * for now, try $ |f(p1) - y|^2 + |f(p2) - y|^2 + |f(p3) - y|^2 $ where $f(p1)$ is model confidence for class y. 
        * this is tricky because the reward isn't calculated over one datapoint any longer but rather a few of them. in addition you have two datasets: the original and the paraphrase. 
        * maybe the best is to compute the paraphrases inside the reward function. or compute them before but just store them, and then reference it in the reward function. 
    * later you can add various terms, e.g. BERTScore, or a term for fluency, or the semantic similarity component. 

**TODO**
* if speed improvement needed - work out how to batch the gen_dataset_paraphrases_simple fn (DONE)
* merge functions togehter 
* fix caching
* might have to bring generating paraphrases into the training loop 

In [6]:
# Precompute paraphrases for the training set and store them
def get_paraphrases(input_text,num_return_sequences,num_beams, num_beam_groups=1,diversity_penalty=0):
    batch = para_tokenizer(input_text,truncation=True,padding='longest', return_tensors="pt").to(device)
    translated = para_model.generate(**batch,num_beams=num_beams, num_return_sequences=num_return_sequences, 
                                   temperature=1.5, num_beam_groups=num_beam_groups, diversity_penalty=diversity_penalty)
    tgt_text = para_tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

# def gen_dataset_paraphrases(x, cname_input, cname_output, n_seed_seqs=32): 
#     """ x: one row of a dataset. 
#     cname_input: column to generate paraphrases for 
#     cname_output: column name to give output of paraphrases 
#     n_seed_seqs: rough indicator of how many paraphrases to return. 
#             For now, keep at 4,8,16,32,64 etc"""
#     # TODO: figure out how to batch this. 
#     if n_seed_seqs % 4 != 0: raise ValueError("keep n_seed_seqs divisible by 4 for now")
#     n = n_seed_seqs/2
#     #low diversity (ld) paraphrases 
#     ld_l = get_paraphrases(x[cname_input],num_return_sequences=int(n),
#                             num_beams=int(n))
#     #high diversity (hd) paraphrases. We can use num_beam_groups and diversity_penalty as hyperparameters. 
#     hd_l =  get_paraphrases(x[cname_input],num_return_sequences=int(n),
#                             num_beams=int(n), num_beam_groups=int(n),diversity_penalty=50002.5)
#     l = ld_l + hd_l 
#     x[cname_output] = l #TODO: change to list(set(l))             
#     return x 


def create_paraphrase_dataset(batch, cname_input, cname_output, n_seed_seqs=32): 
    """Create `n_seed_seq` paraphrases for each example in the batch. Then repeat the other fields 
        so that the resulting datase has the same length as the number of paraphrases. Key assumption is 
        that the same number of paraphrases is created for each example.
    batch: a dict of examples used by the `map` function from the dataset
    cname_input: What column to create paraphrases of 
    cname_output: What to call the column of paraphrases
    n_seed_seqs: Number of paraphrases to generate. """
    
    # Generate paraphrases. 
    # This can be later extended to add diversity or so on. 
    para_l = get_paraphrases(batch[cname_input], n_seed_seqs, n_seed_seqs)
    
    # To return paraphrases as a list of lists for batch input (not done here but might need later)
    #     split_into_sublists = lambda l,n: [l[i:i + n] for i in range(0, len(l), n)]
    #     para_l = split_into_sublists(para_l, n_seed_seqs)
    batch[cname_output] = para_l 
    
    # Repeat each entry in all other columns `n_seed_seq` times so they are the same length
    # as the paraphrase column
    return_d = defaultdict(list) 
    repeat_each_item_n_times = lambda l,n: [o for o in l for i in range(n)]
    for k in batch.keys(): 
        if   k == cname_output: return_d[k] = batch[cname_output]
        else:                   return_d[k] = repeat_each_item_n_times(batch[k], n_seed_seqs)
    return return_d

In [7]:
# Generate paraphrase dataset
n_seed_seqs = 3
cname_input = 'text' # which text column to paraphrase
cname_output= cname_input + '_pphrases'
date = '20210802'
fname = path_cache + '_rt_train'+ date + '_' + str(n_seed_seqs)
if os.path.exists(fname):  
    train_pphrases = datasets.load_from_disk(fname)
else:
    train_pphrases = train.shard(1, 0, contiguous=True)
    # Have to call with batched=True
    # Need to set a batch size otherwise will run out of memory on the GPU card. 
    # 64 seems to work well 
    train_pphrases = train_pphrases.map(
        lambda x: create_paraphrase_dataset(x, n_seed_seqs=n_seed_seqs,
            cname_input=cname_input, cname_output=cname_output),
        batched=True, batch_size=64) 
    train_pphrases_ds.save_to_disk(fname)

NameError: name 'train_pphrases_ds' is not defined

In [8]:
train_pphrases[1:8]

{'label': [1, 1, 1, 1, 1, 1, 1],
 'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .',
  'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .',
  'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot ad

In [ ]:
train_pphrases = train.shard(200, 0, contiguous=True)
train_pphrases = train_pphrases.map(     
        lambda x: create_paraphrase_dataset(x, n_seed_seqs=n_seed_seqs,
            cname_input=cname_input, cname_output=cname_output),
        batched=True, batch_size=4)

In [ ]:
batch={'label': [1, 1, 1, 1], 
       'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .', 'effective but too-tepid biopic', 'if you sometimes like to go to the movies to have fun , wasabi is a good place to start .'], 
       'text_pphrases': ["The rock is going to be the new conan, and he's going to make a bigger splash than arnold schwarzenegger, jean-claud van damme or steven segal.", "The rock is going to be the new conan, and he's going to make a bigger splash than arnold schwarzenegger or jean-claud van damme.", "A column of words can't adequately describe the expanded vision of peter jackson's trilogy.", "A column of words can't adequately describe the expanded vision of Peter Jackson's trilogy.", 'The film was effective but too-tepid.', 'The film is effective but too-tepid.', 'If you like to go to the movies to have fun, you should start at wasabi.', 'If you like to go to the movies to have fun, you can start at wasabi.']
      }
n_seed_seqs = 2
cname_input='text',
cname_output='text_pphrases'

# Only works if the same number of paraphrases is generated for each phrase. 
# Else try something like 
# for o in zip(*batch.values()):
#     d = dict(zip(batch.keys(), o))
#     get_paraphrases(batch[cname_input],num_return_sequences=n_seed_seqs,num_beams=n_seed_seqs)
#     for k,v in d.items(): 
#       return_d[k] += v if k == 'text' else [v for o in range(n_paraphrases)]
# return return_d
return_d = defaultdict(list) 
repeat_each_item_n_times = lambda l,n: [o for o in l for i in range(n)]
for k in batch.keys(): 
    if   k == cname_output: return_d[k] = batch[cname_output]
    else:                   return_d[k] = repeat_each_item_n_times(batch[k], n_seed_seqs)
return return_d




In [ ]:
d

In [ ]:
x={"a":4, "b":[1,2]}

In [ ]:
?x

In [ ]:
n=n_seed_seqs
lambda l,n: [l[i:i + n] for i in range(0, len(l), n)]

In [ ]:
repeat_each_item_n_times(batch['text'], 2)

In [ ]:
batch

In [ ]:
batch.values()